In [13]:
%%capture
pip install iterative-stratification

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn import set_config
import warnings
import optuna
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
from pprint import pprint

warnings.filterwarnings('ignore')

sns.set_theme(style='white',palette='viridis')
pal=sns.color_palette('viridis')

pd.set_option('display.max_rows',100)
set_config(transform_output='pandas')
pd.options.mode.chained_assignment=None



In [15]:
original=pd.read_csv('/home/anushka/Documents/kaggle-Playground-Series/S4E3/Steel Plates Faults.csv',low_memory=False)
original.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0


In [16]:
data=pd.read_csv('/home/anushka/Documents/kaggle-Playground-Series/S4E3/train.csv',low_memory=False)
data.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [17]:
test=pd.read_csv('/home/anushka/Documents/kaggle-Playground-Series/S4E3/test.csv',low_memory=False)
test.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,19219,1015,1033,3826564,3826588,659,23,46,62357,67,...,0.0095,0.5652,1.0000,1.0,2.8410,1.1139,1.6628,0.6727,-0.2261,0.9172
1,19220,1257,1271,419960,419973,370,26,28,39293,92,...,0.0047,0.2414,1.0000,1.0,2.5682,0.9031,1.4472,0.9063,-0.1453,0.9104
2,19221,1358,1372,117715,117724,289,36,32,29386,101,...,0.0155,0.6000,0.7500,0.0,2.4609,1.3222,1.3222,-0.5238,-0.0435,0.6514
3,19222,158,168,232415,232440,80,10,11,8586,107,...,0.0037,0.8000,1.0000,1.0,1.9031,0.6990,1.0414,0.1818,-0.0738,0.2051
4,19223,559,592,544375,544389,140,19,15,15524,103,...,0.0158,0.8421,0.5333,0.0,2.1461,1.3222,1.1461,-0.5714,-0.0894,0.4170


In [18]:
data=pd.concat([data,original],axis=0).drop(columns=['id'],axis=1).drop_duplicates().dropna()
data.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,584,590,909972,909977,16,8,5,2274,113,140,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21160 entries, 0 to 1940
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   X_Minimum              21160 non-null  int64  
 1   X_Maximum              21160 non-null  int64  
 2   Y_Minimum              21160 non-null  int64  
 3   Y_Maximum              21160 non-null  int64  
 4   Pixels_Areas           21160 non-null  int64  
 5   X_Perimeter            21160 non-null  int64  
 6   Y_Perimeter            21160 non-null  int64  
 7   Sum_of_Luminosity      21160 non-null  int64  
 8   Minimum_of_Luminosity  21160 non-null  int64  
 9   Maximum_of_Luminosity  21160 non-null  int64  
 10  Length_of_Conveyer     21160 non-null  int64  
 11  TypeOfSteel_A300       21160 non-null  int64  
 12  TypeOfSteel_A400       21160 non-null  int64  
 13  Steel_Plate_Thickness  21160 non-null  int64  
 14  Edges_Index            21160 non-null  float64
 15  Emp

In [44]:
from tqdm.notebook import tqdm

In [79]:
class Model:

    # initialize the class
    def __init__(self,train,test):
        self.train=train
        self.test=test
        self.model_dict={}
        self.test_predict_list=[]

    def fit(self):
        target_col = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']
        drop_col=['id']

        train_cols=[col for col in self.train.columns if col not in target_col+drop_col]
        scores=[]
        params={'n_estimators': 1459, 'learning_rate': 0.016001966978257513, 'gamma': 0.10333984713821887, 'subsample': 0.6603818318331486, 'colsample_bytree': 0.34393284013626196, 'max_depth': 11, 'min_child_weight': 13, 'reg_lambda': 1.2101866889871998e-08, 'reg_alpha': 0.0007132285308535626}

        for i in range(3):
            mskf=MultilabelStratifiedKFold(n_splits=25, shuffle=True)
            oof_valid_preds=np.zeros((self.train[train_cols].shape[0],len(target_col)))

            for fold,(train_idx,val_idx) in tqdm(enumerate(mskf.split(self.train[train_cols],self.train[target_col]))):
                X_train,y_train=self.train[train_cols].iloc[train_idx],self.train[target_col].iloc[train_idx]
                X_val,y_val=self.train[train_cols].iloc[val_idx],self.train[target_col].iloc[val_idx]

                model=XGBClassifier(**params,random_state=fold+i)

                model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=30,verbose=False) # verbose=False to avoid printing logs
                valid_preds=model.predict_proba(X_val)
                oof_valid_preds[val_idx]=valid_preds
                test_predict=model.predict_proba(self.test[train_cols])
                self.test_predict_list.append(test_predict)
                score=roc_auc_score(y_val,valid_preds,multi_class='ovr')
                self.model_dict[f'fold_{fold}']=model

            oof_score=roc_auc_score(self.train[target_col],oof_valid_preds,multi_class='ovr')
            print(f"OOF auc score for iteration {i+1} is {oof_score}")
            scores.append(oof_score)
        
        return scores, self.test_predict_list
    
    def objective(self,trial):
        target_col = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']
        drop_col=['id']
        test_predict_list = list()
        model_dict = dict()
        
        train_cols = [col for col in self.train.columns.to_list() if col not in target_col + drop_col]
        scores = list()
        params = {'grow_policy': 'depthwise',
                      'n_estimators': trial.suggest_int('n_estimators', 500, 2000), 
                       'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1), 
                       'gamma': trial.suggest_uniform('gamma', 0.1, 1), 
                       'subsample': trial.suggest_uniform('subsample', 0.5, 1),
                       'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3, 1), 
                       'max_depth': trial.suggest_int('max_depth', 5, 30), 
                       'min_child_weight': trial.suggest_int('min_child_weight', 1, 25), 
                       'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10),
                       'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10),
                       'booster':'gbtree',
                       'verbosity':0,
                       'tree_method': 'hist'
                        }
        for i in range(1):
            mskf=MultilabelStratifiedKFold(n_splits=10,shuffle=True)
            oof_valid_preds = np.zeros((self.train[train_cols].shape[0], len(target_col)))
            for fold, (train_idx, valid_idx) in tqdm(enumerate(mskf.split(self.train[train_cols], self.train[target_col]))):
                X_train, y_train = self.train[train_cols].iloc[train_idx], self.train[target_col].iloc[train_idx]
                X_valid, y_valid = self.train[train_cols].iloc[valid_idx], self.train[target_col].iloc[valid_idx]
                
                model = XGBClassifier(random_state=i+fold,**params)
                    
                model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=30,verbose=False)
                    
                valid_preds = model.predict_proba(X_valid)
                oof_valid_preds[valid_idx] = valid_preds
                test_predict = model.predict_proba(self.test[train_cols])
                test_predict_list.append(test_predict)
                score = roc_auc_score(y_valid, valid_preds, multi_class="ovr")
                model_dict[f'fold_{fold}'] = model
                    
            oof_score = roc_auc_score(self.train[target_col], oof_valid_preds, multi_class="ovr")
            #print(f"The OOF auc score for iteration {i+1} is {oof_score}")
            scores.append(oof_score)
        return np.mean(np.array(scores))    
    
    def find_params(self):
        study=optuna.create_study(direction='maximize')
        study.optimize(self.objective,500)
        best_params=study.best_params
        return best_params


In [80]:
model=Model(data,test)
# model.find_params()

In [81]:

scores,preds=model.fit()
print(f"The average OOF roc-auc score is {np.mean(scores)}")


0it [00:00, ?it/s]

OOF auc score for iteration 1 is 0.8967482543923284


0it [00:00, ?it/s]

OOF auc score for iteration 2 is 0.8969307694858608


0it [00:00, ?it/s]

OOF auc score for iteration 3 is 0.8967384163646724
The average OOF roc-auc score is 0.8968058134142872


In [82]:
predictions=np.mean(preds,axis=0)
submit=pd.DataFrame(predictions,columns=['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults'])

submit['id']=test['id']
submit.to_csv('submission1.csv',index=False)
submit

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,id
0,0.562680,0.002523,0.005249,0.000507,0.019372,0.167300,0.378742,19219
1,0.306912,0.026057,0.004119,0.000656,0.161322,0.129804,0.348632,19220
2,0.002415,0.029577,0.037706,0.001086,0.006636,0.289339,0.495585,19221
3,0.135620,0.005217,0.001234,0.002039,0.010215,0.461840,0.436428,19222
4,0.003347,0.006467,0.001554,0.003890,0.008653,0.652768,0.317740,19223
...,...,...,...,...,...,...,...,...
12809,0.070385,0.070421,0.003382,0.000528,0.014695,0.206446,0.431643,32028
12810,0.134489,0.003716,0.018376,0.009033,0.102151,0.201695,0.397015,32029
12811,0.001254,0.001110,0.924348,0.000643,0.001081,0.001544,0.073257,32030
12812,0.395835,0.011185,0.020959,0.000530,0.068324,0.131893,0.308801,32031
